In [ ]:
# TFM_05_normalization_and_generate_models.ipynb

In [ ]:
import pandas as pd
import numpy as np
import time
import logging
import multiprocessing
import warnings
import joblib
import json
import math
import hashlib
from collections import Counter
import gc

from sklearn.model_selection import TimeSeriesSplit, train_test_split, cross_validate
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import (roc_auc_score, accuracy_score, precision_score, recall_score, f1_score,
                             mean_squared_error, mean_absolute_error, r2_score, make_scorer,
                             average_precision_score)
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import (RandomForestClassifier, RandomForestRegressor,
                              GradientBoostingClassifier, GradientBoostingRegressor)
from sklearn.base import clone
from sklearn.exceptions import ConvergenceWarning

try:
    from xgboost import XGBClassifier, XGBRegressor
    XGB_AVAILABLE = True
except ImportError:
    XGB_AVAILABLE = False
    XGBClassifier, XGBRegressor = None, None

try:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout
    from tensorflow.keras.callbacks import EarlyStopping
    from tensorflow.keras import backend as K
    TF_KERAS_AVAILABLE = True
except ImportError:
    TF_KERAS_AVAILABLE = False
    Sequential, LSTM, Dense, Dropout, EarlyStopping, K = None, None, None, None, None, None

try:
    from statsmodels.tsa.arima.model import ARIMA
    from statsmodels.tsa.holtwinters import SimpleExpSmoothing
    STATSMODELS_AVAILABLE = True
except ImportError:
    STATSMODELS_AVAILABLE = False
    ARIMA, SimpleExpSmoothing = None, None

from tqdm import tqdm

# --- Configuraciones ---
pd.set_option('display.float_format', lambda x: '{:.12f}'.format(x))
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 0)

# --- Logging ---
LOG_FILENAME = 'training_full_v2.log'
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - [%(filename)s:%(lineno)d] - %(message)s',
    handlers=[
        logging.FileHandler(LOG_FILENAME, mode='w', encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logging.info(f"Starting training script v2. Log: {LOG_FILENAME}")

# --- Ignore Warnings ---
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", message="Precision is ill-defined and being set to 0.0 due to no predicted samples.")
warnings.filterwarnings("ignore", message="Recall is ill-defined and being set to 0.0 due to no true samples.")

In [ ]:
df_train = pd.read_csv("datos_divididos/datos_financieros_train_completo.csv")
df_test = pd.read_csv("datos_divididos/datos_financieros_test_completo.csv")

objectives = ['target','future_max_increase_capped']

In [3]:
df_train.sample(3)

,Unnamed: 0,timestamp,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,close_lag_1,volume_lag_1,open_lag_1,high_lag_1,low_lag_1,quote_asset_volume_lag_1,number_of_trades_lag_1,taker_buy_base_asset_volume_lag_1,taker_buy_quote_asset_volume_lag_1,close_lag_2,volume_lag_2,open_lag_2,high_lag_2,low_lag_2,quote_asset_volume_lag_2,number_of_trades_lag_2,taker_buy_base_asset_volume_lag_2,taker_buy_quote_asset_volume_lag_2,close_lag_3,volume_lag_3,open_lag_3,high_lag_3,low_lag_3,quote_asset_volume_lag_3,number_of_trades_lag_3,taker_buy_base_asset_volume_lag_3,taker_buy_quote_asset_volume_lag_3,close_lag_4,volume_lag_4,open_lag_4,high_lag_4,low_lag_4,quote_asset_volume_lag_4,number_of_trades_lag_4,taker_buy_base_asset_volume_lag_4,taker_buy_quote_asset_volume_lag_4,close_lag_5,volume_lag_5,open_lag_5,high_lag_5,low_lag_5,quote_asset_volume_lag_5,number_of_trades_lag_5,taker_buy_base_asset_volume_lag_5,taker_buy_quote_asset_volume_lag_5,close_lag_6,volume_lag_6,open_lag_6,high_lag_6,low_lag_6,quote_asset_volume_lag_6,number_of_trades_lag_6,taker_buy_base_asset_volume_lag_6,taker_buy_quote_asset_volume_lag_6,close_lag_7,volume_lag_7,open_lag_7,high_lag_7,low_lag_7,quote_asset_volume_lag_7,number_of_trades_lag_7,taker_buy_base_asset_volume_lag_7,taker_buy_quote_asset_volume_lag_7,close_lag_8,volume_lag_8,open_lag_8,high_lag_8,low_lag_8,quote_asset_volume_lag_8,number_of_trades_lag_8,taker_buy_base_asset_volume_lag_8,taker_buy_quote_asset_volume_lag_8,close_lag_9,volume_lag_9,open_lag_9,high_lag_9,low_lag_9,quote_asset_volume_lag_9,number_of_trades_lag_9,taker_buy_base_asset_volume_lag_9,taker_buy_quote_asset_volume_lag_9,close_lag_10,volume_lag_10,open_lag_10,high_lag_10,low_lag_10,quote_asset_volume_lag_10,number_of_trades_lag_10,taker_buy_base_asset_volume_lag_10,taker_buy_quote_asset_volume_lag_10,close_lag_11,volume_lag_11,open_lag_11,high_lag_11,low_lag_11,quote_asset_volume_lag_11,number_of_trades_lag_11,taker_buy_base_asset_volume_lag_11,taker_buy_quote_asset_volume_lag_11,close_lag_12,volume_lag_12,open_lag_12,high_lag_12,low_lag_12,quote_asset_volume_lag_12,number_of_trades_lag_12,taker_buy_base_asset_volume_lag_12,taker_buy_quote_asset_volume_lag_12,close_lag_13,volume_lag_13,open_lag_13,high_lag_13,low_lag_13,quote_asset_volume_lag_13,number_of_trades_lag_13,taker_buy_base_asset_volume_lag_13,taker_buy_quote_asset_volume_lag_13,close_lag_14,volume_lag_14,open_lag_14,high_lag_14,low_lag_14,quote_asset_volume_lag_14,number_of_trades_lag_14,taker_buy_base_asset_volume_lag_14,taker_buy_quote_asset_volume_lag_14,close_lag_15,volume_lag_15,open_lag_15,high_lag_15,low_lag_15,quote_asset_volume_lag_15,number_of_trades_lag_15,taker_buy_base_asset_volume_lag_15,taker_buy_quote_asset_volume_lag_15,close_lag_16,volume_lag_16,open_lag_16,high_lag_16,low_lag_16,quote_asset_volume_lag_16,number_of_trades_lag_16,taker_buy_base_asset_volume_lag_16,taker_buy_quote_asset_volume_lag_16,close_lag_17,volume_lag_17,open_lag_17,high_lag_17,low_lag_17,quote_asset_volume_lag_17,number_of_trades_lag_17,taker_buy_base_asset_volume_lag_17,taker_buy_quote_asset_volume_lag_17,close_lag_18,volume_lag_18,open_lag_18,high_lag_18,low_lag_18,quote_asset_volume_lag_18,number_of_trades_lag_18,taker_buy_base_asset_volume_lag_18,taker_buy_quote_asset_volume_lag_18,close_lag_19,volume_lag_19,open_lag_19,high_lag_19,low_lag_19,quote_asset_volume_lag_19,number_of_trades_lag_19,taker_buy_base_asset_volume_lag_19,taker_buy_quote_asset_volume_lag_19,close_lag_20,volume_lag_20,open_lag_20,high_lag_20,low_lag_20,quote_asset_volume_lag_20,number_of_trades_lag_20,taker_buy_base_asset_volume_lag_20,taker_buy_quote_asset_volume_lag_20,close_lag_21,volume_lag_21,open_lag_21,high_lag_21,low_lag_21,quote_asset_volume_lag_21,number_of_trades_lag_21,taker_buy_base_asset_volume_lag_21,taker_buy_quote_asset_volume_lag_21,close_lag_22,volume_lag_22,open_lag_22,high_lag_22,low_lag_22,quote_asset_volume_lag_22,n

In [ ]:
drop_cols_initial = ['timestamp', 'Unnamed: 0']
df_train = df_train.drop(columns=[col for col in drop_cols_initial if col in df_train.columns])
df_test = df_test.drop(columns=[col for col in drop_cols_initial if col in df_test.columns])

print("Initial columns dropped.")
print("Train shape:", df_train.shape)
print("Test shape:", df_test.shape)

print("Resetting indices...")
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
print("Indices reset.")

Columnas eliminadas inicialmente.
Train shape: (85932, 441)
Test shape: (21483, 441)
Reseteando índices de df_train y df_test...
Índices reseteados.


In [ ]:
duplicados_train = df_train.duplicated().sum()
print(f"Duplicates in train: {duplicados_train}")

if duplicados_train > 0:
    df_train.drop_duplicates(inplace=True)
    print("Train duplicates removed.")
    print("New train shape:", df_train.shape)

duplicados_test = df_test.duplicated().sum()
print(f"Duplicates in test: {duplicados_test}")

if duplicados_test > 0:
    df_test.drop_duplicates(inplace=True)
    print("Test duplicates removed.")
    print("New test shape:", df_test.shape)

Duplicados en entrenamiento: 0
Duplicados en prueba: 0


In [ ]:
def normalize_column_within_group(col: pd.Series, margen: float = 0.2):
    """
    Purpose: Calculate normalization scaler for a column.
    Args:
        col (pd.Series): Input data.
        margen (float): Margin for scaling.
    Returns:
        tuple: (temporarily scaled column, (min_val_ext, max_val_ext)).
    """
    min_val = col.min()
    max_val = col.max()
    if pd.isna(min_val) or pd.isna(max_val):
        return col, (np.nan, np.nan)

    col_range = max_val - min_val
    if abs(col_range) < 1e-9:
        min_ext = min_val - 0.0001
        max_ext = max_val + 0.0001
        denominador = max_ext - min_ext
        if abs(denominador) < 1e-9:
             scaled_col_temp = pd.Series(0.5, index=col.index)
             scaled_col_temp = scaled_col_temp.where(col.notna(), np.nan)
             return scaled_col_temp, (min_ext, max_ext)
    else:
        min_ext = min_val - margen * col_range
        max_ext = max_val + margen * col_range
        denominador = max_ext - min_ext
        if abs(denominador) < 1e-9:
            scaled_col_temp = pd.Series(0.5, index=col.index)
            scaled_col_temp = scaled_col_temp.where(col.notna(), np.nan)
            return scaled_col_temp, (min_ext, max_ext)

    scaled_col = (col - min_ext) / denominador
    scaled_col = scaled_col.where(col.notna(), np.nan)
    return scaled_col, (min_ext, max_ext)


def normalizar_datos_ml(df_train: pd.DataFrame,
                        df_test: pd.DataFrame,
                        columna_simbolo: str,
                        margen: float = 0.2):
    """
    Purpose: Normalize train/test data grouped by a symbol column.
    Args:
        df_train (pd.DataFrame): Training data.
        df_test (pd.DataFrame): Test data.
        columna_simbolo (str): Grouping column name.
        margen (float): Margin for scaling.
    Returns:
        tuple: (df_train_normalized, df_test_normalized, scalers).
    """
    print(f"Normalizing data for ML by group '{columna_simbolo}'...")

    if columna_simbolo not in df_train.columns or columna_simbolo not in df_test.columns:
        raise ValueError(f"Column '{columna_simbolo}' does not exist in both DataFrames.")

    train_original_idx = df_train.index
    test_original_idx = df_test.index

    if train_original_idx.has_duplicates or test_original_idx.has_duplicates:
         print("Warning: Duplicate indices detected. Resetting recommended.")

    print("\nCalculating scalers on train set...")
    escaladores = {}
    columnas_numericas_train = df_train.select_dtypes(include=np.number).columns.tolist()
    if columna_simbolo in columnas_numericas_train:
        columnas_numericas_train.remove(columna_simbolo)

    if not columnas_numericas_train:
        print("Warning: No numeric columns in df_train to calculate scalers.")
        return df_train.copy(), df_test.copy(), {}

    print(f"Columns for scalers: {columnas_numericas_train}")
    grouped_train = df_train.groupby(columna_simbolo)

    for simbolo, grupo_df_train in tqdm(grouped_train, total=len(grouped_train.groups), desc="Calculating scalers"):
        escaladores_simbolo = {}
        for col_name in columnas_numericas_train:
            if col_name in grupo_df_train.columns:
                try:
                    _ , escalador = normalize_column_within_group(grupo_df_train[col_name], margen)
                    escaladores_simbolo[col_name] = escalador
                except Exception as e_calc:
                    print(f"\nError calculating scaler for {simbolo}/{col_name}: {e_calc}")
                    escaladores_simbolo[col_name] = (np.nan, np.nan)
            else:
                escaladores_simbolo[col_name] = (np.nan, np.nan)
        escaladores[simbolo] = escaladores_simbolo
    print("Scalers calculated.")

    print("\nApplying scalers...")

    def _aplicar_escaladores_interna(df_input: pd.DataFrame, escaladores_dict: dict, desc: str) -> pd.DataFrame:
        df_output = df_input.copy()
        columnas_numericas_df = df_output.select_dtypes(include=np.number).columns.tolist()
        if columna_simbolo in columnas_numericas_df:
            columnas_numericas_df.remove(columna_simbolo)

        grouped_df = df_output.groupby(columna_simbolo)

        for simbolo, grupo_df in tqdm(grouped_df, total=len(grouped_df.groups), desc=desc):
            if simbolo not in escaladores_dict:
                continue

            scalers_for_symbol = escaladores_dict[simbolo]

            for col_name in columnas_numericas_df:
                if col_name in grupo_df.columns and col_name in scalers_for_symbol:
                    min_ext, max_ext = scalers_for_symbol[col_name]

                    if pd.isna(min_ext) or pd.isna(max_ext):
                        df_output.loc[grupo_df.index, col_name] = np.nan
                        continue

                    denominador = max_ext - min_ext
                    col_original = grupo_df[col_name]

                    if abs(denominador) < 1e-9:
                        scaled_col = pd.Series(0.5, index=col_original.index, name=col_name)
                    else:
                        scaled_col = (col_original - min_ext) / denominador

                    scaled_col = scaled_col.where(col_original.notna(), np.nan)
                    df_output.loc[grupo_df.index, col_name] = scaled_col

        return df_output

    df_train_normalizado = _aplicar_escaladores_interna(df_train, escaladores, "Normalizing Train")
    print("Train set normalized.")

    df_test_normalizado = _aplicar_escaladores_interna(df_test, escaladores, "Normalizing Test")
    print("Test set normalized.")

    print("\nNormalization complete.")
    df_train_normalizado.index = train_original_idx
    df_test_normalizado.index = test_original_idx
    return df_train_normalizado, df_test_normalizado, escaladores

In [ ]:
def guardar_escaladores(escaladores, nombre_archivo):
    """
    Purpose: Save scalers to a file.
    Args:
        escaladores (dict): Scalers data.
        nombre_archivo (str): Filename.
    Returns:
        None.
    """
    joblib.dump(escaladores, nombre_archivo)
    print(f"Scalers saved to {nombre_archivo}")

def cargar_escaladores(nombre_archivo):
    """
    Purpose: Load scalers from a file.
    Args:
        nombre_archivo (str): Filename.
    Returns:
        dict: Loaded scalers.
    """
    escaladores = joblib.load(nombre_archivo)
    print(f"Scalers loaded from {nombre_archivo}")
    return escaladores

In [ ]:
def convert_value_for_json(value):
    """
    Purpose: Convert NumPy/NaN values to JSON-compatible types.
    Args:
        value: Input value.
    Returns:
        JSON-compatible value.
    """
    if isinstance(value, np.ndarray):
         return [convert_value_for_json(v) for v in value]
    if isinstance(value, (np.number, np.bool_)):
        if np.isnan(value):
             return None
        return value.item()
    if isinstance(value, float) and math.isnan(value):
         return None
    return value

def guardar_escaladores_joblib_json(escaladores, base_nombre_archivo):
    """
    Purpose: Save scalers in joblib and JSON formats.
    Args:
        escaladores (dict): Scaler dictionary.
        base_nombre_archivo (str): Base filename without extension.
    Returns:
        None.
    """
    joblib_filename = f"{base_nombre_archivo}.joblib"
    try:
        joblib.dump(escaladores, joblib_filename)
        print(f"Scalers saved in Joblib format: {joblib_filename}")
    except Exception as e_joblib:
        print(f"Error saving scalers in Joblib: {e_joblib}")

    json_filename = f"{base_nombre_archivo}.json"
    json_compatible_scalers = {}
    try:
        for simbolo, cols_scalers in escaladores.items():
            json_compatible_scalers[simbolo] = {}
            for col, (min_e, max_e) in cols_scalers.items():
                 json_min = convert_value_for_json(min_e)
                 json_max = convert_value_for_json(max_e)
                 json_compatible_scalers[simbolo][col] = [json_min, json_max]

        with open(json_filename, 'w', encoding='utf-8') as f:
            json.dump(json_compatible_scalers, f, indent=4, ensure_ascii=False)
        print(f"Scalers saved in JSON format: {json_filename}")

    except Exception as e_json:
        print(f"Error saving scalers in JSON: {e_json}")

In [ ]:
columna_simbolo = 'symbol'

df_train_normalizado, df_test_normalizado, escaladores = normalizar_datos_ml(
    df_train, df_test, columna_simbolo, margen=0.2
)

guardar_escaladores(escaladores, 'escaladores.joblib')
guardar_escaladores_joblib_json(escaladores, 'escaladores_final')

Normalizando para ML (fit en train, transform train/test) por grupo 'symbol'...

--- Calculando Escaladores en Train Set ---
Columnas para escaladores: ['open', 'high', 'low', 'close', 'volume', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'close_lag_1', 'volume_lag_1', 'open_lag_1', 'high_lag_1', 'low_lag_1', 'quote_asset_volume_lag_1', 'number_of_trades_lag_1', 'taker_buy_base_asset_volume_lag_1', 'taker_buy_quote_asset_volume_lag_1', 'close_lag_2', 'volume_lag_2', 'open_lag_2', 'high_lag_2', 'low_lag_2', 'quote_asset_volume_lag_2', 'number_of_trades_lag_2', 'taker_buy_base_asset_volume_lag_2', 'taker_buy_quote_asset_volume_lag_2', 'close_lag_3', 'volume_lag_3', 'open_lag_3', 'high_lag_3', 'low_lag_3', 'quote_asset_volume_lag_3', 'number_of_trades_lag_3', 'taker_buy_base_asset_volume_lag_3', 'taker_buy_quote_asset_volume_lag_3', 'close_lag_4', 'volume_lag_4', 'open_lag_4', 'high_lag_4', 'low_lag_4', 'quote_asset_volume_lag_4

Calculando escaladores: 100%|██████████| 77/77 [00:13<00:00,  5.91it/s]


Escaladores calculados.

--- Aplicando Escaladores ---


Normalizando Train:   0%|          | 0/77 [00:00<?, ?it/s]C:\Users\forad\AppData\Local\Temp\ipykernel_12436\2676540166.py:128: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.85714286 0.14285714 0.14285714 ... 0.85714286 0.14285714 0.14285714]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_output.loc[grupo_df.index, col_name] = scaled_col
C:\Users\forad\AppData\Local\Temp\ipykernel_12436\2676540166.py:128: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.14285714 0.14285714 0.14285714 ... 0.14285714 0.14285714 0.14285714]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_output.loc[grupo_df.index, col_name] = scaled_col
C:\Users\forad\AppData\Local\Temp\ipykernel_12436\2676540166.py:128: FutureWarning: Setting an item of incompatible 

df_train normalizado.


Normalizando Test:   0%|          | 0/77 [00:00<?, ?it/s]C:\Users\forad\AppData\Local\Temp\ipykernel_12436\2676540166.py:128: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.85714286 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714 0.14285714 0.85714286 0.14285714 0.14285714 0.14285714
 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714 0.85714286 0.85714286 0.14285714 0.14285714 0.14285714
 0.85714286 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714 0.14285714 0.14285714 0.85714286 0.85714286 0.14285714
 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714 0.14285714 0.85714286 0.14285714 0.14285714 0.14285714
 0.14285714 0.85714286 0.14285714 0.14285714 0.14285714 0.14285714
 0.85714286 0.14285714 0.14285714 0.14285714 0.14285714 0.85714286
 0.14285

df_test normalizado.

Normalización completada (calculado en train, aplicado en train/test).
Escaladores guardados en escaladores.joblib
Escaladores guardados en formato Joblib: escaladores_final.joblib
Escaladores guardados en formato JSON: escaladores_final.json


In [10]:
df_train_normalizado.sample(5)

,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,close_lag_1,volume_lag_1,open_lag_1,high_lag_1,low_lag_1,quote_asset_volume_lag_1,number_of_trades_lag_1,taker_buy_base_asset_volume_lag_1,taker_buy_quote_asset_volume_lag_1,close_lag_2,volume_lag_2,open_lag_2,high_lag_2,low_lag_2,quote_asset_volume_lag_2,number_of_trades_lag_2,taker_buy_base_asset_volume_lag_2,taker_buy_quote_asset_volume_lag_2,close_lag_3,volume_lag_3,open_lag_3,high_lag_3,low_lag_3,quote_asset_volume_lag_3,number_of_trades_lag_3,taker_buy_base_asset_volume_lag_3,taker_buy_quote_asset_volume_lag_3,close_lag_4,volume_lag_4,open_lag_4,high_lag_4,low_lag_4,quote_asset_volume_lag_4,number_of_trades_lag_4,taker_buy_base_asset_volume_lag_4,taker_buy_quote_asset_volume_lag_4,close_lag_5,volume_lag_5,open_lag_5,high_lag_5,low_lag_5,quote_asset_volume_lag_5,number_of_trades_lag_5,taker_buy_base_asset_volume_lag_5,taker_buy_quote_asset_volume_lag_5,close_lag_6,volume_lag_6,open_lag_6,high_lag_6,low_lag_6,quote_asset_volume_lag_6,number_of_trades_lag_6,taker_buy_base_asset_volume_lag_6,taker_buy_quote_asset_volume_lag_6,close_lag_7,volume_lag_7,open_lag_7,high_lag_7,low_lag_7,quote_asset_volume_lag_7,number_of_trades_lag_7,taker_buy_base_asset_volume_lag_7,taker_buy_quote_asset_volume_lag_7,close_lag_8,volume_lag_8,open_lag_8,high_lag_8,low_lag_8,quote_asset_volume_lag_8,number_of_trades_lag_8,taker_buy_base_asset_volume_lag_8,taker_buy_quote_asset_volume_lag_8,close_lag_9,volume_lag_9,open_lag_9,high_lag_9,low_lag_9,quote_asset_volume_lag_9,number_of_trades_lag_9,taker_buy_base_asset_volume_lag_9,taker_buy_quote_asset_volume_lag_9,close_lag_10,volume_lag_10,open_lag_10,high_lag_10,low_lag_10,quote_asset_volume_lag_10,number_of_trades_lag_10,taker_buy_base_asset_volume_lag_10,taker_buy_quote_asset_volume_lag_10,close_lag_11,volume_lag_11,open_lag_11,high_lag_11,low_lag_11,quote_asset_volume_lag_11,number_of_trades_lag_11,taker_buy_base_asset_volume_lag_11,taker_buy_quote_asset_volume_lag_11,close_lag_12,volume_lag_12,open_lag_12,high_lag_12,low_lag_12,quote_asset_volume_lag_12,number_of_trades_lag_12,taker_buy_base_asset_volume_lag_12,taker_buy_quote_asset_volume_lag_12,close_lag_13,volume_lag_13,open_lag_13,high_lag_13,low_lag_13,quote_asset_volume_lag_13,number_of_trades_lag_13,taker_buy_base_asset_volume_lag_13,taker_buy_quote_asset_volume_lag_13,close_lag_14,volume_lag_14,open_lag_14,high_lag_14,low_lag_14,quote_asset_volume_lag_14,number_of_trades_lag_14,taker_buy_base_asset_volume_lag_14,taker_buy_quote_asset_volume_lag_14,close_lag_15,volume_lag_15,open_lag_15,high_lag_15,low_lag_15,quote_asset_volume_lag_15,number_of_trades_lag_15,taker_buy_base_asset_volume_lag_15,taker_buy_quote_asset_volume_lag_15,close_lag_16,volume_lag_16,open_lag_16,high_lag_16,low_lag_16,quote_asset_volume_lag_16,number_of_trades_lag_16,taker_buy_base_asset_volume_lag_16,taker_buy_quote_asset_volume_lag_16,close_lag_17,volume_lag_17,open_lag_17,high_lag_17,low_lag_17,quote_asset_volume_lag_17,number_of_trades_lag_17,taker_buy_base_asset_volume_lag_17,taker_buy_quote_asset_volume_lag_17,close_lag_18,volume_lag_18,open_lag_18,high_lag_18,low_lag_18,quote_asset_volume_lag_18,number_of_trades_lag_18,taker_buy_base_asset_volume_lag_18,taker_buy_quote_asset_volume_lag_18,close_lag_19,volume_lag_19,open_lag_19,high_lag_19,low_lag_19,quote_asset_volume_lag_19,number_of_trades_lag_19,taker_buy_base_asset_volume_lag_19,taker_buy_quote_asset_volume_lag_19,close_lag_20,volume_lag_20,open_lag_20,high_lag_20,low_lag_20,quote_asset_volume_lag_20,number_of_trades_lag_20,taker_buy_base_asset_volume_lag_20,taker_buy_quote_asset_volume_lag_20,close_lag_21,volume_lag_21,open_lag_21,high_lag_21,low_lag_21,quote_asset_volume_lag_21,number_of_trades_lag_21,taker_buy_base_asset_volume_lag_21,taker_buy_quote_asset_volume_lag_21,close_lag_22,volume_lag_22,open_lag_22,high_lag_22,low_lag_22,quote_asset_volume_lag_22,number_of_trades_lag_2

In [11]:
df_test_normalizado.sample(5)

,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,close_lag_1,volume_lag_1,open_lag_1,high_lag_1,low_lag_1,quote_asset_volume_lag_1,number_of_trades_lag_1,taker_buy_base_asset_volume_lag_1,taker_buy_quote_asset_volume_lag_1,close_lag_2,volume_lag_2,open_lag_2,high_lag_2,low_lag_2,quote_asset_volume_lag_2,number_of_trades_lag_2,taker_buy_base_asset_volume_lag_2,taker_buy_quote_asset_volume_lag_2,close_lag_3,volume_lag_3,open_lag_3,high_lag_3,low_lag_3,quote_asset_volume_lag_3,number_of_trades_lag_3,taker_buy_base_asset_volume_lag_3,taker_buy_quote_asset_volume_lag_3,close_lag_4,volume_lag_4,open_lag_4,high_lag_4,low_lag_4,quote_asset_volume_lag_4,number_of_trades_lag_4,taker_buy_base_asset_volume_lag_4,taker_buy_quote_asset_volume_lag_4,close_lag_5,volume_lag_5,open_lag_5,high_lag_5,low_lag_5,quote_asset_volume_lag_5,number_of_trades_lag_5,taker_buy_base_asset_volume_lag_5,taker_buy_quote_asset_volume_lag_5,close_lag_6,volume_lag_6,open_lag_6,high_lag_6,low_lag_6,quote_asset_volume_lag_6,number_of_trades_lag_6,taker_buy_base_asset_volume_lag_6,taker_buy_quote_asset_volume_lag_6,close_lag_7,volume_lag_7,open_lag_7,high_lag_7,low_lag_7,quote_asset_volume_lag_7,number_of_trades_lag_7,taker_buy_base_asset_volume_lag_7,taker_buy_quote_asset_volume_lag_7,close_lag_8,volume_lag_8,open_lag_8,high_lag_8,low_lag_8,quote_asset_volume_lag_8,number_of_trades_lag_8,taker_buy_base_asset_volume_lag_8,taker_buy_quote_asset_volume_lag_8,close_lag_9,volume_lag_9,open_lag_9,high_lag_9,low_lag_9,quote_asset_volume_lag_9,number_of_trades_lag_9,taker_buy_base_asset_volume_lag_9,taker_buy_quote_asset_volume_lag_9,close_lag_10,volume_lag_10,open_lag_10,high_lag_10,low_lag_10,quote_asset_volume_lag_10,number_of_trades_lag_10,taker_buy_base_asset_volume_lag_10,taker_buy_quote_asset_volume_lag_10,close_lag_11,volume_lag_11,open_lag_11,high_lag_11,low_lag_11,quote_asset_volume_lag_11,number_of_trades_lag_11,taker_buy_base_asset_volume_lag_11,taker_buy_quote_asset_volume_lag_11,close_lag_12,volume_lag_12,open_lag_12,high_lag_12,low_lag_12,quote_asset_volume_lag_12,number_of_trades_lag_12,taker_buy_base_asset_volume_lag_12,taker_buy_quote_asset_volume_lag_12,close_lag_13,volume_lag_13,open_lag_13,high_lag_13,low_lag_13,quote_asset_volume_lag_13,number_of_trades_lag_13,taker_buy_base_asset_volume_lag_13,taker_buy_quote_asset_volume_lag_13,close_lag_14,volume_lag_14,open_lag_14,high_lag_14,low_lag_14,quote_asset_volume_lag_14,number_of_trades_lag_14,taker_buy_base_asset_volume_lag_14,taker_buy_quote_asset_volume_lag_14,close_lag_15,volume_lag_15,open_lag_15,high_lag_15,low_lag_15,quote_asset_volume_lag_15,number_of_trades_lag_15,taker_buy_base_asset_volume_lag_15,taker_buy_quote_asset_volume_lag_15,close_lag_16,volume_lag_16,open_lag_16,high_lag_16,low_lag_16,quote_asset_volume_lag_16,number_of_trades_lag_16,taker_buy_base_asset_volume_lag_16,taker_buy_quote_asset_volume_lag_16,close_lag_17,volume_lag_17,open_lag_17,high_lag_17,low_lag_17,quote_asset_volume_lag_17,number_of_trades_lag_17,taker_buy_base_asset_volume_lag_17,taker_buy_quote_asset_volume_lag_17,close_lag_18,volume_lag_18,open_lag_18,high_lag_18,low_lag_18,quote_asset_volume_lag_18,number_of_trades_lag_18,taker_buy_base_asset_volume_lag_18,taker_buy_quote_asset_volume_lag_18,close_lag_19,volume_lag_19,open_lag_19,high_lag_19,low_lag_19,quote_asset_volume_lag_19,number_of_trades_lag_19,taker_buy_base_asset_volume_lag_19,taker_buy_quote_asset_volume_lag_19,close_lag_20,volume_lag_20,open_lag_20,high_lag_20,low_lag_20,quote_asset_volume_lag_20,number_of_trades_lag_20,taker_buy_base_asset_volume_lag_20,taker_buy_quote_asset_volume_lag_20,close_lag_21,volume_lag_21,open_lag_21,high_lag_21,low_lag_21,quote_asset_volume_lag_21,number_of_trades_lag_21,taker_buy_base_asset_volume_lag_21,taker_buy_quote_asset_volume_lag_21,close_lag_22,volume_lag_22,open_lag_22,high_lag_22,low_lag_22,quote_asset_volume_lag_22,number_of_trades_lag_2

In [ ]:
objetivos = ['target', 'future_max_increase_capped']

X_train = df_train_normalizado.copy()
y_train = df_train[objetivos[0]].copy()
Z_train = df_train[objetivos[1]].copy()

X_test = df_test_normalizado.copy()
y_test = df_test[objetivos[0]].copy()
Z_test = df_test[objetivos[1]].copy()

print("Train and test sets created.")
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)
print("Z_train shape:", Z_train.shape)
print("Z_test shape:", Z_test.shape)

Conjuntos de entrenamiento y prueba creados.
X_train shape: (85932, 441)
X_test shape: (21483, 441)
y_train shape: (85932,)
y_test shape: (21483,)
Z_train shape: (85932,)
Z_test shape: (21483,)


In [ ]:
X_train_s = X_train.drop(columns=['symbol', 'target', 'future_max_increase_capped'], axis=1, errors='ignore')
X_test_s = X_test.drop(columns=['symbol', 'target', 'future_max_increase_capped'], axis=1, errors='ignore')

print("Training classification model ('target', superlow)...")
with tqdm(total=100, desc="Classification (superlow)", position=0, leave=True) as pbar:
    modelo_clasificacion = RandomForestClassifier(random_state=42, n_estimators=4, max_depth=2, class_weight='balanced')
    modelo_clasificacion.fit(X_train_s, y_train)
    pbar.update(50)

    y_pred_proba = modelo_clasificacion.predict_proba(X_test_s)[:, 1]
    pbar.update(25)
    auc = roc_auc_score(y_test, y_pred_proba)
    pbar.update(12)
    y_pred = modelo_clasificacion.predict(X_test_s)
    pbar.update(8)
    accuracy = accuracy_score(y_test, y_pred)
    pbar.update(5)
    print(f"  AUC: {auc:.4f}")
    print(f"  Accuracy: {accuracy:.4f}")

print("Training regression model ('future_max_increase_capped', superlow)...")
with tqdm(total=100, desc="Regression (superlow)", position=0, leave=True) as pbar:
    modelo_regresion = RandomForestRegressor(random_state=42, n_estimators=4, max_depth=2)
    modelo_regresion.fit(X_train_s, Z_train)
    pbar.update(50)
    Z_pred = modelo_regresion.predict(X_test_s)
    pbar.update(30)
    mse = mean_squared_error(Z_test, Z_pred)
    pbar.update(20)
    print(f"  MSE: {mse:.4f}")

del X_train_s, X_test_s

Entrenando modelo de clasificación para 'target' (superlow)...


Clasificación (superlow): 100%|██████████| 100/100 [00:01<00:00, 84.21it/s]


  AUC: 0.8517916157425423
  Accuracy: 0.7835032351161384
Entrenando modelo de regresión para 'future_max_increase_capped' (superlow)...


Regresión (superlow): 100%|██████████| 100/100 [00:21<00:00,  4.70it/s]

  MSE: 0.00010718095123370026


In [ ]:
modelo_clasificacion_default = RandomForestClassifier(random_state=42)
print("Default RandomForestClassifier parameters:")
print(modelo_clasificacion_default.get_params())

modelo_regresion_default = RandomForestRegressor(random_state=42)
print("\nDefault RandomForestRegressor parameters:")
print(modelo_regresion_default.get_params())

Parámetros predeterminados del RandomForestClassifier:
{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}

Parámetros predeterminados del RandomForestRegressor:
{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}


In [ ]:
N_SPLITS_LIST = [3, 5, 7]

PRIMARY_METRIC_CLASSIFICATION = 'f1_weighted'
PRIMARY_METRIC_REGRESSION = 'mse'

METRICS_CLASSIFICATION = {
    'accuracy': True, 'f1_weighted': True, 'precision_weighted': True, 'recall_weighted': True,
    'f1_macro': True, 'precision_macro': True, 'recall_macro': True, 'roc_auc': True,
    # 'average_precision': True
}
METRICS_REGRESSION = {
    'mse': False, 'mae': False, 'rmse': False, 'r2': True,
}

logging.info(f"CV splits to test: {N_SPLITS_LIST}")
logging.info(f"Default Primary Classification Metric: {PRIMARY_METRIC_CLASSIFICATION}")
logging.info(f"Tracked Classification Metrics: {list(METRICS_CLASSIFICATION.keys())}")
logging.info(f"Tracked Regression Metrics: {list(METRICS_REGRESSION.keys())}")
logging.info(f"TensorFlow/Keras Available: {TF_KERAS_AVAILABLE}")
logging.info(f"Statsmodels Available: {STATSMODELS_AVAILABLE}")
logging.info(f"XGBoost Available: {XGB_AVAILABLE}")

# --- Usar los datos cargados previamente ---
X_train_main = X_train.drop(columns=['symbol', 'target', 'future_max_increase_capped'], axis=1, errors='ignore')
X_test_main = X_test.drop(columns=['symbol', 'target', 'future_max_increase_capped'], axis=1, errors='ignore')
y_train_main = y_train.copy()
y_test_main = y_test.copy()
Z_train_main = Z_train.copy()
Z_test_main = Z_test.copy()

logging.info(f"Using loaded data shapes: X_train={X_train_main.shape}, y_train={y_train_main.shape}, Z_train={Z_train_main.shape}")
logging.info(f"Class Distribution in Train: {Counter(y_train_main)}")


scale_pos_weight_value = 1.0
try:
    counts_train = Counter(y_train_main)
    count_neg_train, count_pos_train = counts_train.get(0, 0), counts_train.get(1, 0)
    if count_pos_train > 0 and count_neg_train > 0:
        scale_pos_weight_value = count_neg_train / count_pos_train
        logging.info(f"Calculated scale_pos_weight: {scale_pos_weight_value:.4f} ({count_neg_train} neg / {count_pos_train} pos)")
    elif count_pos_train == 0:
        logging.warning("No positive examples (class 1) found in y_train. Cannot calculate scale_pos_weight, using 1.0.")
    else:
         logging.warning("No negative examples (class 0) found in y_train. Cannot calculate scale_pos_weight, using 1.0.")
except Exception as e:
    logging.warning(f"Error calculating scale_pos_weight: {e}. Using 1.0.")


def seleccionar_caracteristicas_umbral(X_df, y_series, umbral):
    """
    Purpose: Select features based on correlation threshold.
    Args:
        X_df (DataFrame): Input features.
        y_series (Series): Target variable.
        umbral (float): Correlation threshold.
    Returns:
        list: Selected feature names.
    """
    logging.debug(f"Running placeholder feature selection with threshold {umbral}")
    if X_df.empty or y_series.empty: return []
    try:
        y_numeric = y_series.astype(float)
        numeric_cols = X_df.select_dtypes(include=np.number).columns
        if numeric_cols.empty: return []
        correlations = X_df[numeric_cols].corrwith(y_numeric).dropna()
        if correlations.empty: return []
        selected_features = correlations[abs(correlations) >= umbral].index.tolist()
        if not selected_features and not correlations.empty:
             logging.warning(f"No numeric features passed threshold {umbral}. Returning top 5.")
             return correlations.abs().nlargest(min(5, len(correlations))).index.tolist()
        time.sleep(0.05) # Simulate time
        return selected_features
    except Exception as e:
        logging.error(f"Error in placeholder feature selection: {e}")
        return X_df.columns[:10].tolist()


modelos_all = {
    'random_forest_classifier': {
        'model_class': RandomForestClassifier, 'type': 'classifier',
        'param_grid': [
            {'n_estimators': 10, 'max_depth': 3, 'min_samples_leaf': 10},
            {'n_estimators': 25, 'max_depth': 5, 'min_samples_leaf': 7},
            {'n_estimators': 50, 'max_depth': 10, 'min_samples_leaf': 5, 'class_weight': 'balanced'},
            {'n_estimators': 100, 'max_depth': 15, 'min_samples_leaf': 3, 'class_weight': 'balanced'},
            {'n_estimators': 250, 'max_depth': 25, 'min_samples_leaf': 1, 'class_weight': 'balanced', 'max_features': 'sqrt'},
            {'n_estimators': 500, 'max_depth': None, 'min_samples_leaf': 1, 'class_weight': 'balanced', 'max_features': None},
            {'n_estimators': 750, 'max_depth': None, 'min_samples_leaf': 1, 'class_weight': 'balanced', 'max_features': None},
            {'n_estimators': 1000, 'max_depth': None, 'min_samples_leaf': 1, 'class_weight': 'balanced', 'max_features': None},
        ],
        'fixed_params': {'random_state': 42, 'n_jobs': -1},
        'enabled': True, 'umbral': 0.001, 'allow_multiprocessing': False,
    },
    'gradient_boosting_classifier': {
        'model_class': GradientBoostingClassifier, 'type': 'classifier',
        'param_grid': [
            {'n_estimators': 10, 'max_depth': 2, 'learning_rate': 0.2, 'subsample': 1.0},
            {'n_estimators': 30, 'max_depth': 3, 'learning_rate': 0.15, 'subsample': 0.9},
            {'n_estimators': 75, 'max_depth': 4, 'learning_rate': 0.1, 'subsample': 0.8},
            {'n_estimators': 150, 'max_depth': 5, 'learning_rate': 0.05, 'subsample': 0.7},
            {'n_estimators': 300, 'max_depth': 6, 'learning_rate': 0.02, 'subsample': 0.6},
            {'n_estimators': 750, 'max_depth': 7, 'learning_rate': 0.01, 'subsample': 0.5},
            {'n_estimators': 1200, 'max_depth': 8, 'learning_rate': 0.01, 'subsample': 0.5},
            {'n_estimators': 2000, 'max_depth': 9, 'learning_rate': 0.005, 'subsample': 0.5},
        ],
        'fixed_params': {'random_state': 42},
        'enabled': True, 'umbral': 0.001, 'allow_multiprocessing': True,
    },
    'logistic_regression': {
        'model_class': LogisticRegression, 'type': 'classifier',
        'param_grid': [
             {'C': 0.001, 'solver': 'liblinear', 'penalty': 'l2'},
             {'C': 0.01, 'solver': 'liblinear', 'penalty': 'l2'},
             {'C': 0.1, 'solver': 'liblinear', 'penalty': 'l1', 'class_weight': 'balanced'},
             {'C': 1.0, 'solver': 'liblinear', 'penalty': 'l2', 'class_weight': 'balanced'},
             {'C': 10.0, 'solver': 'saga', 'penalty': 'l1', 'class_weight': 'balanced'},
             {'C': 100.0, 'solver': 'saga', 'penalty': 'l2', 'class_weight': 'balanced'},
             {'C': 500.0, 'solver': 'saga', 'penalty': 'l2', 'class_weight': 'balanced'},
             {'C': 1000.0, 'solver': 'saga', 'penalty': 'l2', 'class_weight': 'balanced'},
        ],
        'fixed_params': {'random_state': 42, 'max_iter': 3000},
        'enabled': True, 'umbral': 0.001, 'allow_multiprocessing': True,
    },
    'xgboost_classifier': {
        'model_class': XGBClassifier, 'type': 'classifier',
        'param_grid': [
            {'n_estimators': 10, 'max_depth': 2, 'learning_rate': 0.3, 'scale_pos_weight': scale_pos_weight_value},
            {'n_estimators': 30, 'max_depth': 3, 'learning_rate': 0.2, 'subsample': 0.9, 'colsample_bytree': 0.9, 'gamma': 0.1, 'scale_pos_weight': scale_pos_weight_value},
            {'n_estimators': 75, 'max_depth': 4, 'learning_rate': 0.1, 'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0.3, 'reg_alpha': 0.05, 'scale_pos_weight': scale_pos_weight_value},
            {'n_estimators': 150, 'max_depth': 5, 'learning_rate': 0.05, 'subsample': 0.7, 'colsample_bytree': 0.7, 'gamma': 0.5, 'reg_alpha': 0.1, 'reg_lambda': 0.5, 'scale_pos_weight': scale_pos_weight_value},
            {'n_estimators': 300, 'max_depth': 7, 'learning_rate': 0.02, 'subsample': 0.6, 'colsample_bytree': 0.6, 'gamma': 0.5, 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'scale_pos_weight': scale_pos_weight_value},
            {'n_estimators': 750, 'max_depth': 9, 'learning_rate': 0.01, 'subsample': 0.5, 'colsample_bytree': 0.5, 'gamma': 0.2, 'reg_alpha': 0.05, 'reg_lambda': 0.1, 'scale_pos_weight': scale_pos_weight_value},
            {'n_estimators': 1200, 'max_depth': 11, 'learning_rate': 0.01, 'subsample': 0.5, 'colsample_bytree': 0.5, 'gamma': 0.1, 'reg_alpha': 0, 'reg_lambda': 0.1, 'scale_pos_weight': scale_pos_weight_value},
            {'n_estimators': 2500, 'max_depth': 13, 'learning_rate': 0.005, 'subsample': 0.5, 'colsample_bytree': 0.5, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 0.05, 'scale_pos_weight': scale_pos_weight_value},
        ],
        'fixed_params': {'random_state': 42, 'use_label_encoder': False, 'eval_metric': 'logloss'},
        'enabled': XGB_AVAILABLE, 'umbral': 0.001, 'allow_multiprocessing': True,
    },
    'random_forest_regressor': {
        'model_class': RandomForestRegressor, 'type': 'regressor',
        'param_grid': [
            {'n_estimators': 10, 'max_depth': 3, 'min_samples_leaf': 10},
            {'n_estimators': 25, 'max_depth': 5, 'min_samples_leaf': 7},
            {'n_estimators': 50, 'max_depth': 10, 'min_samples_leaf': 5},
            {'n_estimators': 100, 'max_depth': 15, 'min_samples_leaf': 3},
            {'n_estimators': 250, 'max_depth': 25, 'min_samples_leaf': 1, 'max_features': 'sqrt'},
            {'n_estimators': 500, 'max_depth': None, 'min_samples_leaf': 1, 'max_features': None},
            {'n_estimators': 750, 'max_depth': None, 'min_samples_leaf': 1, 'max_features': None},
            {'n_estimators': 1000, 'max_depth': None, 'min_samples_leaf': 1, 'max_features': None},
        ],
        'fixed_params': {'random_state': 42, 'n_jobs': -1}, 'enabled': True, 'umbral': 0.001, 'allow_multiprocessing': False,
    },
    'gradient_boosting_regressor': {
        'model_class': GradientBoostingRegressor,
        'type': 'regressor',
        'param_grid': [
            {'n_estimators': 1200, 'max_depth': 8, 'learning_rate': 0.01, 'subsample': 0.5},
            {'n_estimators': 2000, 'max_depth': 9, 'learning_rate': 0.005, 'subsample': 0.5},
            {'n_estimators': 3000, 'max_depth': 9, 'learning_rate': 0.005, 'subsample': 0.5},
            {'n_estimators': 5000, 'max_depth': 9, 'learning_rate': 0.002, 'subsample': 0.5},
            {'n_estimators': 7500, 'max_depth': 9, 'learning_rate': 0.002, 'subsample': 0.5},
        ],
        'fixed_params': {'random_state': 42},
        'enabled': True,
        'umbral': 0.001,
        'allow_multiprocessing': True,
    },
    'linear_regression': {
        'model_class': LinearRegression, 'type': 'regressor',
        'param_grid': [
            {'fit_intercept': False}, {'fit_intercept': True},
            {'fit_intercept': False}, {'fit_intercept': True},
            {'fit_intercept': False}, {'fit_intercept': True},
            {'fit_intercept': False}, {'fit_intercept': True},
        ],
        'fixed_params': {}, 'enabled': True, 'umbral': 0.001, 'allow_multiprocessing': True,
    },
    'xgboost_regressor': {
        'model_class': XGBRegressor, 'type': 'regressor',
        'param_grid': [
            {'n_estimators': 10, 'max_depth': 2, 'learning_rate': 0.3},
            {'n_estimators': 30, 'max_depth': 3, 'learning_rate': 0.2, 'subsample': 0.9, 'colsample_bytree': 0.9, 'gamma': 0.1},
            {'n_estimators': 75, 'max_depth': 4, 'learning_rate': 0.1, 'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0.3, 'reg_alpha': 0.05},
            {'n_estimators': 150, 'max_depth': 5, 'learning_rate': 0.05, 'subsample': 0.7, 'colsample_bytree': 0.7, 'gamma': 0.5, 'reg_alpha': 0.1, 'reg_lambda': 0.5},
            {'n_estimators': 300, 'max_depth': 7, 'learning_rate': 0.02, 'subsample': 0.6, 'colsample_bytree': 0.6, 'gamma': 0.7, 'reg_alpha': 0.5, 'reg_lambda': 0.1},
            {'n_estimators': 750, 'max_depth': 9, 'learning_rate': 0.01, 'subsample': 0.5, 'colsample_bytree': 0.5, 'gamma': 0.2, 'reg_alpha': 0.05, 'reg_lambda': 0.1},
            {'n_estimators': 1200, 'max_depth': 11, 'learning_rate': 0.01, 'subsample': 0.5, 'colsample_bytree': 0.5, 'gamma': 0.1, 'reg_alpha': 0, 'reg_lambda': 0.1},
            {'n_estimators': 2500, 'max_depth': 13, 'learning_rate': 0.005, 'subsample': 0.5, 'colsample_bytree': 0.5, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 0.05},
        ],
        'fixed_params': {'random_state': 42, 'objective': 'reg:squarederror'}, 'enabled': XGB_AVAILABLE, 'umbral': 0.001, 'allow_multiprocessing': True,
    },
    'exponential_smoothing': {
        'model_class': None, 'type': 'regressor',
        'param_grid': [
            {'smoothing_level': 0.1, 'optimized': False},
            {'smoothing_level': 0.3, 'optimized': False},
            {'smoothing_level': 0.5, 'optimized': False},
            {'smoothing_level': 0.8, 'optimized': False},
            {'smoothing_level': None, 'optimized': True},
            {'smoothing_level': None, 'optimized': True, 'use_brute': True},
            {'smoothing_level': None, 'optimized': True, 'use_brute': True},
            {'smoothing_level': None, 'optimized': True, 'use_brute': False},
        ],
        'fixed_params': {}, 'enabled': STATSMODELS_AVAILABLE, 'umbral': 0.001, 'requires_timeseries': True,
    },
    'lstm_classifier': {
        'model_class': None, 'type': 'classifier',
        'param_grid': [
            {'epochs': 5, 'batch_size': 128, 'patience': 3, 'dropout': 0.1, 'units': 30},
            {'epochs': 10, 'batch_size': 64, 'patience': 5, 'dropout': 0.2, 'units': 50},
            {'epochs': 25, 'batch_size': 64, 'patience': 7, 'dropout': 0.3, 'units': 50},
            {'epochs': 50, 'batch_size': 32, 'patience': 10, 'dropout': 0.4, 'units': 75},
            {'epochs': 100, 'batch_size': 32, 'patience': 15, 'dropout': 0.5, 'units': 75},
            {'epochs': 200, 'batch_size': 16, 'patience': 20, 'dropout': 0.5, 'units': 100},
            {'epochs': 300, 'batch_size': 16, 'patience': 30, 'dropout': 0.5, 'units': 120},
            {'epochs': 500, 'batch_size': 8, 'patience': 40, 'dropout': 0.5, 'units': 150},
        ],
        'fixed_params': {}, 'enabled': TF_KERAS_AVAILABLE, 'umbral': 0.001, 'requires_timeseries': True, 'timesteps': 10,
    },
    'lstm_regressor': {
        'model_class': None, 'type': 'regressor',
        'param_grid': [
            {'epochs': 5, 'batch_size': 128, 'patience': 3, 'dropout': 0.1, 'units': 30},
            {'epochs': 10, 'batch_size': 64, 'patience': 5, 'dropout': 0.2, 'units': 50},
            {'epochs': 25, 'batch_size': 64, 'patience': 7, 'dropout': 0.3, 'units': 50},
            {'epochs': 50, 'batch_size': 32, 'patience': 10, 'dropout': 0.4, 'units': 75},
            {'epochs': 100, 'batch_size': 32, 'patience': 15, 'dropout': 0.5, 'units': 75},
            {'epochs': 200, 'batch_size': 16, 'patience': 20, 'dropout': 0.5, 'units': 100},
            {'epochs': 300, 'batch_size': 16, 'patience': 30, 'dropout': 0.5, 'units': 120},
            {'epochs': 500, 'batch_size': 8, 'patience': 40, 'dropout': 0.5, 'units': 150},
        ],
        'fixed_params': {}, 'enabled': TF_KERAS_AVAILABLE, 'umbral': 0.001, 'requires_timeseries': True, 'timesteps': 10,
    }
}

TQDM_DETAIL_LEVEL = 'param'

In [ ]:
def calculate_metrics(model_type, y_true, y_pred, y_prob=None):
    """
    Purpose: Calculate evaluation metrics.
    Args:
        model_type (str): 'classifier' or 'regressor'.
        y_true (array-like): True target values.
        y_pred (array-like): Predicted values or classes.
        y_prob (array-like, optional): Predicted probabilities (for classifier).
    Returns:
        dict: Calculated metrics.
    """
    metrics = {}
    if y_true is None or y_pred is None: return metrics

    try:
        y_true = np.asarray(y_true).flatten()
        y_pred = np.asarray(y_pred).flatten()
        if len(y_true) == 0 or len(y_pred) == 0: return metrics
        if len(y_true) != len(y_pred):
             min_len = min(len(y_true), len(y_pred))
             logging.warning(f"Different lengths in calculate_metrics. Truncating to {min_len}.")
             y_true, y_pred = y_true[:min_len], y_pred[:min_len]

        y_prob_for_auc = None
        if y_prob is not None:
            y_prob = np.asarray(y_prob)
            if len(y_prob) != len(y_true):
                if len(y_prob) > len(y_true): y_prob = y_prob[:len(y_true)]
                else: y_prob = None
            if y_prob is not None:
                if y_prob.ndim == 2 and y_prob.shape[1] == 2: y_prob_for_auc = y_prob[:, 1]
                elif y_prob.ndim == 1: y_prob_for_auc = y_prob
                elif y_prob.ndim == 2 and y_prob.shape[1] == 1: y_prob_for_auc = y_prob.flatten()
                else: logging.warning(f"y_prob format ({y_prob.shape}) not recognized for AUC."); y_prob_for_auc = None
        elif len(np.unique(y_pred)) > 2 and np.all((y_pred >= 0) & (y_pred <= 1)):
            y_prob_for_auc = y_pred

    except Exception as e:
        logging.error(f"Error preparing arrays in calculate_metrics: {e}")
        return metrics

    try:
        if model_type == 'classifier':
            threshold = 0.5
            y_true_int = y_true.astype(int)

            if y_prob_for_auc is not None:
                y_pred_class = (y_prob_for_auc >= threshold).astype(int)
            elif np.issubdtype(y_pred.dtype, np.number) and not np.issubdtype(y_pred.dtype, np.integer):
                y_pred_class = np.round(y_pred).astype(int)
            else:
                y_pred_class = y_pred.astype(int)
            y_pred_for_metrics = y_pred_class

            unique_true = np.unique(y_true_int)
            n_classes = len(unique_true)

            try: metrics['accuracy'] = accuracy_score(y_true_int, y_pred_for_metrics)
            except Exception: pass

            if n_classes > 1:
                common_args = {'y_true': y_true_int, 'y_pred': y_pred_for_metrics, 'zero_division': 0}
                try: metrics['f1_weighted'] = f1_score(**common_args, average='weighted')
                except Exception: pass
                try: metrics['precision_weighted'] = precision_score(**common_args, average='weighted')
                except Exception: pass
                try: metrics['recall_weighted'] = recall_score(**common_args, average='weighted')
                except Exception: pass
                try: metrics['f1_macro'] = f1_score(**common_args, average='macro')
                except Exception: pass
                try: metrics['precision_macro'] = precision_score(**common_args, average='macro')
                except Exception: pass
                try: metrics['recall_macro'] = recall_score(**common_args, average='macro')
                except Exception: pass

                if y_prob_for_auc is not None:
                     try: metrics['roc_auc'] = roc_auc_score(y_true_int, y_prob_for_auc)
                     except ValueError: logging.debug("Ignoring ROC AUC: Only one class?")
                     except Exception: pass
                else: metrics['roc_auc'] = None
            else:
                 logging.debug("Only one class in y_true, calculating only accuracy.")
                 for k in METRICS_CLASSIFICATION:
                     if k != 'accuracy': metrics[k] = None

        elif model_type == 'regressor':
            try: metrics['mse'] = mean_squared_error(y_true, y_pred)
            except Exception: pass
            try: metrics['mae'] = mean_absolute_error(y_true, y_pred)
            except Exception: pass
            try:
                if 'mse' in metrics and metrics['mse'] is not None and metrics['mse'] >= 0:
                    metrics['rmse'] = np.sqrt(metrics['mse'])
            except Exception: pass
            try: metrics['r2'] = r2_score(y_true, y_pred)
            except Exception: pass
        else:
            logging.error(f"Unknown model type in calculate_metrics: {model_type}")

    except Exception as e:
        logging.error(f"General error calculating metrics: {e}", exc_info=True)

    final_metrics = {}
    expected_keys = list(METRICS_CLASSIFICATION.keys()) if model_type == 'classifier' else list(METRICS_REGRESSION.keys())
    for key in expected_keys:
        value = metrics.get(key)
        if isinstance(value, (float, np.number)):
            if pd.isna(value) or np.isinf(value): final_metrics[key] = None
            else: final_metrics[key] = round(value, 5)
        else: final_metrics[key] = value

    return final_metrics


resultados = {}
modelos = modelos_all # Use the full dictionary defined previously

model_progress = tqdm(modelos.items(), desc="Models", total=len(modelos), position=0, leave=True,
                      disable=(TQDM_DETAIL_LEVEL not in ['modelo', 'param', 'split']))

for nombre_modelo, config_modelo in model_progress:
    model_progress.set_description(f"Model: {nombre_modelo}")

    if not config_modelo.get('enabled', True):
        logging.info(f"Skipping disabled model: {nombre_modelo}")
        continue

    model_class = config_modelo.get('model_class')
    is_special_ts = nombre_modelo in ['arima', 'exponential_smoothing']
    is_lstm = 'lstm' in nombre_modelo.lower()

    if not model_class and not is_special_ts and not is_lstm:
         logging.error(f"Invalid config for {nombre_modelo}. Skipping.")
         continue

    if is_lstm and not TF_KERAS_AVAILABLE: logging.warning(f"Skipping {nombre_modelo}: TF/Keras not available."); continue
    if is_special_ts and not STATSMODELS_AVAILABLE: logging.warning(f"Skipping {nombre_modelo}: Statsmodels not available."); continue
    if 'xgboost' in nombre_modelo.lower() and not XGB_AVAILABLE: logging.warning(f"Skipping {nombre_modelo}: XGBoost not available."); continue

    logging.info(f"--- Processing: {nombre_modelo} ---")

    try:
        X_train_base, X_test_base = X_train_main.copy(), X_test_main.copy()
        y_train_base = y_train_main.copy() if config_modelo['type'] == 'classifier' else Z_train_main.copy()
        y_test_base = y_test_main.copy() if config_modelo['type'] == 'classifier' else Z_test_main.copy()
    except Exception as e: logging.error(f"[{nombre_modelo}] Error copying data: {e}. Skipping."); continue

    X_train_actual, X_test_actual = X_train_base, X_test_base
    y_train_actual, y_test_actual = y_train_base, y_test_base
    caracteristicas_originales = X_train_main.columns.tolist()
    caracteristicas_seleccionadas = caracteristicas_originales

    requires_ts = config_modelo.get('requires_timeseries', False)
    realizar_seleccion = not requires_ts and config_modelo['type'] != 'regressor'
    if realizar_seleccion and 'umbral' in config_modelo:
        try:
            logging.info(f"[{nombre_modelo}] Performing feature selection...")
            target_seleccion = y_train_actual
            caracteristicas_seleccionadas_temp = seleccionar_caracteristicas_umbral(
                X_train_actual, target_seleccion, config_modelo['umbral']
            )
            if not caracteristicas_seleccionadas_temp:
                 logging.warning(f"[{nombre_modelo}] No features selected! Using all.")
            else:
                caracteristicas_seleccionadas = list(caracteristicas_seleccionadas_temp)
                X_train_actual = X_train_actual[caracteristicas_seleccionadas]
                X_test_actual = X_test_actual[caracteristicas_seleccionadas]
                logging.info(f"[{nombre_modelo}] # Selected features: {len(caracteristicas_seleccionadas)}")
        except Exception as e:
            logging.error(f"[{nombre_modelo}] Feature selection error: {e}. Using all.", exc_info=False) # Hide traceback for learner
            caracteristicas_seleccionadas = caracteristicas_originales
            X_train_actual, X_test_actual = X_train_base, X_test_base
    else:
         logging.info(f"[{nombre_modelo}] Skipping feature selection, using {len(caracteristicas_seleccionadas)} features.")

    X_train_eval, X_test_eval = X_train_actual, X_test_actual
    y_train_eval, y_test_eval = y_train_actual, y_test_actual
    X_val_reshaped, y_val_reshaped = None, None
    class_weights_dict = None

    if is_lstm:
        try:
            logging.info(f"[{nombre_modelo}] Performing LSTM preprocessing...")
            time_steps = config_modelo.get('timesteps', 10); assert time_steps > 0
            X_train_lstm, X_val_lstm, y_train_lstm, y_val_lstm = train_test_split(
                X_train_actual, y_train_actual, test_size=0.1, random_state=42, shuffle=False
            )
            scaler = MinMaxScaler(feature_range=(0, 1))
            X_train_lstm_numeric = X_train_lstm.select_dtypes(include=np.number)
            if X_train_lstm_numeric.empty: raise ValueError("No numeric data to scale.")
            scaler.fit(X_train_lstm_numeric)
            X_train_scaled_df, X_val_scaled_df, X_test_scaled_df = X_train_lstm.copy(), X_val_lstm.copy(), X_test_actual.copy()
            num_cols = X_train_lstm_numeric.columns
            X_train_scaled_df[num_cols] = scaler.transform(X_train_lstm_numeric)
            X_val_scaled_df[num_cols] = scaler.transform(X_val_lstm.select_dtypes(include=np.number))
            X_test_scaled_df[num_cols] = scaler.transform(X_test_actual.select_dtypes(include=np.number))
            X_train_scaled, X_val_scaled, X_test_scaled = X_train_scaled_df.values, X_val_scaled_df.values, X_test_scaled_df.values

            def create_dataset(X, y, time_steps=1):
                 if len(X) < time_steps: return np.array([]), np.array([])
                 Xs, ys = [], []
                 y_arr = y.values if isinstance(y, pd.Series) else np.asarray(y)
                 for i in range(len(X) - time_steps + 1):
                     Xs.append(X[i:(i + time_steps), :]); ys.append(y_arr[i + time_steps - 1])
                 return np.array(Xs), np.array(ys)

            X_train_reshaped, y_train_reshaped = create_dataset(X_train_scaled, y_train_lstm, time_steps)
            X_val_reshaped, y_val_reshaped = create_dataset(X_val_scaled, y_val_lstm, time_steps)
            X_test_reshaped, y_test_reshaped = create_dataset(X_test_scaled, y_test_actual, time_steps)

            if X_train_reshaped.size == 0 or X_test_reshaped.size == 0 or X_val_reshaped.size == 0:
                logging.warning(f"[{nombre_modelo}] Could not create LSTM sequences. Skipping model."); continue

            X_train_eval, y_train_eval = X_train_reshaped, y_train_reshaped
            X_test_eval, y_test_eval = X_test_reshaped, y_test_reshaped
            logging.info(f"[{nombre_modelo}] LSTM data shapes: Train{X_train_eval.shape}, Val{X_val_reshaped.shape}, Test{X_test_eval.shape}")

            if config_modelo['type'] == 'classifier':
                try:
                    counts_lstm_train = Counter(y_train_eval)
                    if len(counts_lstm_train) > 1:
                         majority_lstm = max(counts_lstm_train.values())
                         class_weights_dict = {cls: float(majority_lstm/count) for cls, count in counts_lstm_train.items() if count > 0}
                         logging.debug(f"[{nombre_modelo}] Calculated LSTM class weights: {class_weights_dict}")
                    else: logging.warning(f"[{nombre_modelo}] Only one class in LSTM y_train_eval.")
                except Exception as e_cw_lstm:
                    logging.warning(f"[{nombre_modelo}] Error calculating LSTM class weights: {e_cw_lstm}.")
        except Exception as e:
            logging.error(f"[{nombre_modelo}] LSTM preprocessing error: {e}", exc_info=False); continue # Hide traceback

    elif is_special_ts:
         try:
            logging.info(f"[{nombre_modelo}] Preparing data for TS models...")
            y_train_eval = y_train_actual.values.flatten() if hasattr(y_train_actual, 'values') else np.asarray(y_train_actual).flatten()
            y_test_eval = y_test_actual.values.flatten() if hasattr(y_test_actual, 'values') else np.asarray(y_test_actual).flatten()
            logging.info(f"[{nombre_modelo}] TS Target Data: Train len={len(y_train_eval)}, Test len={len(y_test_eval)}")
         except Exception as e:
             logging.error(f"[{nombre_modelo}] Error preparing TS data: {e}", exc_info=False); continue # Hide traceback

    split_progress = tqdm(N_SPLITS_LIST, desc=f"Splits ({nombre_modelo})", position=1, leave=False, disable=(TQDM_DETAIL_LEVEL not in ['split']))
    for n_splits in split_progress:
        split_progress.set_postfix_str(f"n_splits={n_splits}", refresh=True)

        param_progress = tqdm(config_modelo.get('param_grid', [{}]), desc=f"Params ({nombre_modelo}, splits={n_splits})", position=2, leave=False, disable=(TQDM_DETAIL_LEVEL not in ['param', 'split']))
        parametros_ejecutados = False
        for params in param_progress:
            parametros_ejecutados = True
            todos_los_parametros = {**config_modelo.get('fixed_params', {}), **params}
            param_progress.set_postfix_str(str(params), refresh=True)

            modelo = None
            resultado_actual = {
                'params': todos_los_parametros, 'features': caracteristicas_seleccionadas,
                'n_features_used': len(caracteristicas_seleccionadas), 'n_splits': n_splits,
                'config': {k: v for k, v in config_modelo.items() if k not in ['model_class', 'param_grid', 'fixed_params']},
                'train_time_mean': None, 'predict_time_mean': None, 'cv_scores': {},
                'test_metrics': {}, 'status': 'Pending', 'error_message': None
            }
            param_str_sorted = str(sorted(params.items()))
            result_key = f"{nombre_modelo}_splits_{n_splits}_params_{param_str_sorted}"

            try:
                logging.debug(f"[{result_key}] Creating instance...")
                if model_class and not is_special_ts:
                    if 'xgboost' in nombre_modelo.lower() and 'n_jobs' not in todos_los_parametros:
                        n_cv_jobs = 1 if not config_modelo.get('allow_multiprocessing', True) else multiprocessing.cpu_count()
                        if n_cv_jobs > 1: todos_los_parametros['n_jobs'] = 1
                    modelo = model_class(**todos_los_parametros)
                elif is_lstm:
                    if not TF_KERAS_AVAILABLE: raise ImportError("TF/Keras not available")
                    logging.debug(f"[{result_key}] Clearing Keras session...")
                    if K: K.clear_session(); gc.collect()
                    if X_train_eval.ndim != 3: raise ValueError(f"LSTM data must be 3D, got {X_train_eval.ndim}")
                    input_shape = (X_train_eval.shape[1], X_train_eval.shape[2])
                    param_hash = hashlib.md5(param_str_sorted.encode('utf-8')).hexdigest()[:8]
                    clean_nombre_modelo = nombre_modelo.replace('-', '_').replace(' ', '_')
                    model_unique_name = f"{clean_nombre_modelo}_{n_splits}_{param_hash}"
                    modelo = Sequential(name=model_unique_name)
                    lstm_units = todos_los_parametros.get('units', 50)
                    modelo.add(LSTM(units=lstm_units, input_shape=input_shape, return_sequences=False))
                    dropout_rate = todos_los_parametros.get('dropout', 0.0)
                    if dropout_rate > 0: modelo.add(Dropout(dropout_rate))
                    activation = 'sigmoid' if config_modelo['type'] == 'classifier' else 'linear'
                    modelo.add(Dense(1, activation=activation))
                    loss = 'binary_crossentropy' if config_modelo['type'] == 'classifier' else 'mean_squared_error'
                    optimizer = todos_los_parametros.get('optimizer', 'adam')
                    modelo.compile(optimizer=optimizer, loss=loss)
                    logging.debug(f"[{model_unique_name}] Keras model compiled.")
                elif is_special_ts: pass
                else: raise ValueError(f"Could not create model {nombre_modelo}")

                logging.debug(f"[{result_key}] Training/Evaluating...")
                if is_special_ts or is_lstm:
                    y_pred_test = None; fit_time = 0; pred_time = 0; model_fit = None
                    start_fit = time.time()
                    if nombre_modelo == 'arima':
                        order = todos_los_parametros.get('order')
                        if not order or len(order) != 3: raise ValueError("Invalid ARIMA order")
                        if len(y_train_eval) < sum(order) + 1: raise ValueError("Insufficient data")
                        model_fit = ARIMA(y_train_eval, order=order).fit()
                    elif nombre_modelo == 'exponential_smoothing':
                        init_params = config_modelo.get('fixed_params', {})
                        fit_params = params
                        allowed_fit_params = ['smoothing_level', 'optimized', 'use_brute']
                        actual_fit_params = {k: v for k, v in fit_params.items() if k in allowed_fit_params}
                        model_fit = SimpleExpSmoothing(y_train_eval, **init_params).fit(**actual_fit_params)
                    elif is_lstm:
                        if X_train_eval is None or y_train_eval is None or X_val_reshaped is None or y_val_reshaped is None:
                             raise ValueError("LSTM data not ready.")
                        epochs = todos_los_parametros.get('epochs', 10)
                        batch_size = todos_los_parametros.get('batch_size', 32)
                        patience = todos_los_parametros.get('patience', 5)
                        es = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True, verbose=0, mode='min')
                        history = modelo.fit(X_train_eval, y_train_eval, epochs=epochs, batch_size=batch_size,
                                             validation_data=(X_val_reshaped, y_val_reshaped), verbose=0, callbacks=[es],
                                             class_weight=class_weights_dict if config_modelo['type'] == 'classifier' and class_weights_dict else None
                                             )
                        model_fit = modelo
                    else: raise NotImplementedError(f"Direct training for {nombre_modelo} not implemented.")
                    fit_time = time.time() - start_fit
                    resultado_actual['train_time_mean'] = round(fit_time, 5)

                    start_pred = time.time()
                    if nombre_modelo in ['arima', 'exponential_smoothing']:
                        n_steps = len(y_test_eval)
                        y_pred_test = model_fit.forecast(steps=n_steps) if n_steps > 0 else np.array([])
                    elif is_lstm:
                        y_pred_test = model_fit.predict(X_test_eval, verbose=0, batch_size=todos_los_parametros.get('batch_size', 32)) if X_test_eval is not None and len(X_test_eval) > 0 else np.array([])
                    pred_time = time.time() - start_pred
                    resultado_actual['predict_time_mean'] = round(pred_time, 5)

                    y_prob_test = y_pred_test if is_lstm and config_modelo['type'] == 'classifier' else None
                    test_metrics_calc = calculate_metrics(config_modelo['type'], y_test_eval, y_pred_test, y_prob_test)
                    resultado_actual['test_metrics'] = test_metrics_calc if test_metrics_calc else {}
                    logging.info(f"    [{result_key}] -> Test Metrics: {resultado_actual['test_metrics']}")
                    resultado_actual['cv_scores'] = {'info': 'CV not applicable'}

                else:
                    cv = TimeSeriesSplit(n_splits=n_splits)
                    use_multiprocessing_cv = config_modelo.get('allow_multiprocessing', True)
                    model_handles_parallel = 'n_jobs' in modelo.get_params() and modelo.get_params()['n_jobs'] != 1
                    n_jobs_cv = multiprocessing.cpu_count() if use_multiprocessing_cv and not model_handles_parallel else 1
                    logging.debug(f"[{result_key}] Starting CV (n_jobs={n_jobs_cv})...")

                    scoring_dict = {}
                    if config_modelo['type'] == 'classifier':
                        scoring_dict = {
                            'accuracy': make_scorer(accuracy_score),
                            'f1_weighted': make_scorer(f1_score, average='weighted', zero_division=0),
                            'precision_weighted': make_scorer(precision_score, average='weighted', zero_division=0),
                            'recall_weighted': make_scorer(recall_score, average='weighted', zero_division=0),
                            'f1_macro': make_scorer(f1_score, average='macro', zero_division=0),
                            'precision_macro': make_scorer(precision_score, average='macro', zero_division=0),
                            'recall_macro': make_scorer(recall_score, average='macro', zero_division=0),
                        }
                        if hasattr(modelo, 'predict_proba'): scoring_dict['roc_auc'] = 'roc_auc'
                        # if hasattr(modelo, 'predict_proba'):
                        #     scoring_dict['average_precision'] = make_scorer(average_precision_score, needs_proba=True)

                    elif config_modelo['type'] == 'regressor':
                        scoring_dict = {
                            'mse': make_scorer(mean_squared_error, greater_is_better=False),
                            'mae': make_scorer(mean_absolute_error, greater_is_better=False),
                            'rmse': make_scorer(lambda y, p: np.sqrt(mean_squared_error(y, p)), greater_is_better=False),
                            'r2': 'r2'
                        }

                    cv_results = cross_validate(modelo, X_train_eval, y_train_eval,
                                                cv=cv, scoring=scoring_dict,
                                                n_jobs=n_jobs_cv, return_train_score=False,
                                                error_score='raise')

                    resultado_actual['train_time_mean'] = round(np.nanmean(cv_results.get('fit_time', [np.nan])), 5)
                    resultado_actual['predict_time_mean'] = round(np.nanmean(cv_results.get('score_time', [np.nan])), 5)
                    cv_scores_summary = {}
                    for metric_key in cv_results:
                        if metric_key.startswith('test_'):
                            clean_metric_name = metric_key[5:]
                            scores = cv_results[metric_key]
                            if config_modelo['type'] == 'regressor' and clean_metric_name in ['mse', 'mae', 'rmse']:
                                scores = -scores
                            mean_score = np.nanmean(scores) if np.any(~np.isnan(scores)) else np.nan
                            std_score = np.nanstd(scores) if np.any(~np.isnan(scores)) else np.nan
                            cv_scores_summary[clean_metric_name] = mean_score
                            cv_scores_summary[clean_metric_name + '_std'] = std_score
                    resultado_actual['cv_scores'] = { k: (round(v, 5) if not pd.isna(v) else None) for k, v in cv_scores_summary.items() }
                    logging.info(f"    [{result_key}] -> CV Mean Scores: {resultado_actual['cv_scores']}")

                    try:
                        logging.debug(f"    [{result_key}] Re-training on full train set...")
                        final_model = clone(modelo).fit(X_train_eval, y_train_eval)
                        y_pred_test_final = final_model.predict(X_test_eval)
                        y_prob_test_final = None
                        if config_modelo['type'] == 'classifier':
                            if hasattr(final_model, 'predict_proba'):
                                try: y_prob_test_final = final_model.predict_proba(X_test_eval)
                                except Exception as proba_err: logging.debug(f"Error final predict_proba: {proba_err}")
                            elif hasattr(final_model, 'decision_function'):
                                try: y_prob_test_final = final_model.decision_function(X_test_eval)
                                except Exception as df_err: logging.debug(f"Error final decision_function: {df_err}")

                        test_metrics_calc = calculate_metrics(config_modelo['type'], y_test_eval, y_pred_test_final, y_prob_test_final)
                        resultado_actual['test_metrics'] = test_metrics_calc if test_metrics_calc else {}
                        logging.info(f"    [{result_key}] -> Final Test Metrics: {resultado_actual['test_metrics']}")
                    except Exception as e_final:
                         logging.warning(f"    [{result_key}] Error in final re-train/evaluation: {e_final}")
                         resultado_actual['test_metrics'] = {'error': f'Re-train/Eval Error: {str(e_final)}'}

                resultado_actual['status'] = 'Completed'

            except (MemoryError, ImportError, Exception) as e:
                error_type = type(e).__name__
                resultado_actual['status'] = f'Error ({error_type})'
                resultado_actual['error_message'] = str(e)
                logging.error(f"[{result_key}] Processing error: {e}", exc_info=False) # Hide traceback
                if is_lstm and 'K' in locals() and K is not None: K.clear_session()
                gc.collect()
                if isinstance(e, (MemoryError, ImportError)):
                    param_progress.close(); split_progress.close(); break
            finally:
                param_str_sorted_final = str(sorted(params.items()))
                final_result_key = f"{nombre_modelo}_splits_{n_splits}_params_{param_str_sorted_final}"
                resultados[final_result_key] = resultado_actual
                if 'modelo' in locals() and modelo is not None: del modelo
                if 'model_fit' in locals() and model_fit is not None: del model_fit
                if 'final_model' in locals() and final_model is not None: del final_model
                if 'history' in locals(): del history
                gc.collect()

            if resultado_actual.get('status', '').startswith('Error (Import') or \
               resultado_actual.get('status', '').startswith('Error (Memory'):
                break
        if resultado_actual.get('status', '').startswith('Error (Import') or \
           resultado_actual.get('status', '').startswith('Error (Memory'):
            break
        if not parametros_ejecutados:
             logging.warning(f"[{nombre_modelo}, splits={n_splits}] No parameters were executed.")


logging.info("\n--- Identifying Best Models per Metric ---")
best_results_clf = {metric: {'key': None, 'score': -np.inf if higher_is_better else np.inf}
                    for metric, higher_is_better in METRICS_CLASSIFICATION.items()}
best_results_reg = {metric: {'key': None, 'score': -np.inf if higher_is_better else np.inf}
                    for metric, higher_is_better in METRICS_REGRESSION.items()}

for key, resultado in resultados.items():
    if resultado.get('status') != 'Completed': continue
    model_type = resultado['config']['type']
    metrics_source = None; score_origin = "N/A"

    if 'test_metrics' in resultado and resultado['test_metrics'] and 'error' not in resultado['test_metrics']:
        metrics_source = resultado['test_metrics']; score_origin = "Test Set"
    elif 'cv_scores' in resultado and resultado['cv_scores'] and 'info' not in resultado['cv_scores']:
         metrics_source = resultado['cv_scores']; score_origin = "CV Mean"
    else: continue

    metrics_to_check = METRICS_CLASSIFICATION if model_type == 'classifier' else METRICS_REGRESSION
    best_results_dict = best_results_clf if model_type == 'classifier' else best_results_reg

    for metric, higher_is_better in metrics_to_check.items():
        current_score = metrics_source.get(metric, None)
        if current_score is None or pd.isna(current_score) or np.isinf(current_score): continue

        best_so_far = best_results_dict[metric]['score']
        is_better = (higher_is_better and current_score > best_so_far) or \
                    (not higher_is_better and current_score < best_so_far)
        if is_better:
            best_results_dict[metric].update({'score': current_score, 'key': key, 'source': score_origin})


print("\n" + "="*30 + " Final Results per Metric " + "="*30)
def print_result_details(metric_name, best_info, all_results):
    """
    Purpose: Print formatted results for a metric.
    Args:
        metric_name (str): Name of the metric.
        best_info (dict): Dictionary containing best score and key.
        all_results (dict): Dictionary containing all experiment results.
    Returns:
        None.
    """
    key, score, source = best_info.get('key'), best_info.get('score'), best_info.get('source', 'N/A')
    print(f"\n--- Best for '{metric_name.upper()}' (Score: {score:.5f}, Source: {source}) ---")
    if not key or key not in all_results: print("  No valid model found."); return
    res = all_results[key]
    try: model_name_base = key.split('_splits_')[0]
    except Exception: model_name_base = key
    print(f"  Key: {key}")
    print(f"  Model: {model_name_base} | Params: {res.get('params', 'N/A')} | Splits: {res.get('n_splits', 'N/A')}")
    print(f"  CV Mean: {res.get('cv_scores', 'N/A')}")
    print(f"  Test Final: {res.get('test_metrics', 'N/A')}")
    print(f"  Features: {res.get('n_features_used', 'N/A')}")

print("\n" + "*"*15 + " Classification " + "*"*15)
found_any_clf = any(info['key'] for info in best_results_clf.values())
if found_any_clf:
    for metric, best_info in best_results_clf.items():
        if best_info['key']:
             print_result_details(metric, best_info, resultados)
else: print("\nNo valid classification results found.")

print("\n" + "*"*15 + " Regression " + "*"*15)
found_any_reg = any(info['key'] for info in best_results_reg.values())
if found_any_reg:
    for metric, best_info in best_results_reg.items():
         if best_info['key']:
            print_result_details(metric, best_info, resultados)
else: print("\nNo valid regression results found.")

print("\n" + "="*78)


def default_serializer(obj):
    """
    Purpose: JSON serializer for complex types (NumPy, Timestamps, etc.).
    Args:
        obj: Object to serialize.
    Returns:
        Serializable object or string representation.
    """
    if isinstance(obj, (np.integer)): return int(obj)
    elif isinstance(obj, (np.floating)): return float(obj) if not pd.isna(obj) and not np.isinf(obj) else None
    elif isinstance(obj, (np.ndarray,)): return obj.tolist() if obj.size < 100 else f"<ndarray shape={obj.shape}>"
    elif isinstance(obj, (np.bool_)): return bool(obj)
    elif isinstance(obj, pd.Timestamp): return obj.isoformat()
    elif isinstance(obj, pd.Index): return obj.tolist()
    elif isinstance(obj, type): return str(obj)
    elif obj is pd.NA or pd.isna(obj): return None
    try: json.dumps(obj); return obj
    except TypeError: return str(obj)

output_filename = 'resultados_modelos_detallado_v4_multi_metric.json'
logging.info(f"Attempting to save detailed results to '{output_filename}'...")
try:
    with open(output_filename, 'w') as f:
        json.dump(resultados, f, indent=4, default=default_serializer)
    logging.info(f"Detailed results saved successfully to '{output_filename}'")
except Exception as e:
    logging.error(f"Could not save JSON results: {e}", exc_info=False) # Hide traceback

logging.info("\nProcess completed.")

Models:   0%|          | 0/11 [00:00<?, ?it/s]

Params (random_forest_classifier, splits=3):   0%|          | 0/6 [00:00<?, ?it/s]

Params (random_forest_classifier, splits=5):   0%|          | 0/6 [00:00<?, ?it/s]

Params (random_forest_classifier, splits=7):   0%|          | 0/6 [00:00<?, ?it/s]

Params (gradient_boosting_classifier, splits=3):   0%|          | 0/6 [00:00<?, ?it/s]

...

Params (lstm_classifier, splits=7):   0%|          | 0/6 [00:00<?, ?it/s]

Params (lstm_regressor, splits=3):   0%|          | 0/6 [00:00<?, ?it/s]

Params (lstm_regressor, splits=5):   0%|          | 0/6 [00:00<?, ?it/s]

Params (lstm_regressor, splits=7):   0%|          | 0/6 [00:00<?, ?it/s]


============================== Resultats Finals ==============================

🏆 Millor Model de Classificació:
  🔑 Key Completa: lstm_classifier_splits_3_params_[('batch_size', 128), ('dropout', 0.1), ('epochs', 5), ('patience', 3)]
  🤖 Model Base: lstm_classifier
  ⚙️ Paràmetres: {'units': 50, 'epochs': 5, 'batch_size': 128, 'patience': 3, 'dropout': 0.1}
  #️⃣ Splits CV/TS: 3
  🌟 Score Primari (AUC): 0.80860 (Font: Test Set)
  📊 Métriques CV mitjanes: {'info': 'CV no aplicable per a aquest model'}
  🧪 Métriques Test Finals: {'accuracy': 0.97382, 'f1': 0.0, 'precision': 0.0, 'recall': 0.0, 'auc': np.float64(0.8086)}
  #️⃣ Features Usades: 500
     Features (500): ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9']... (490 més)
  ⏱️ T. Entr/Pred Mitjà (s): 2.039 / 0.281
  ⓘ Status Execució: Completat

🏆 Millor Model de Regressió:
  🔑 Key Completa: gradient_boosting_regressor_splits_3_params_[('learning_ra